In [9]:
import yfinance as yf
from pathlib import Path
import yaml

from src.data_loader import extract_field_yf, prepare_prices
from src.data_loader import compute_returns, save_parquet

In [10]:
params = yaml.safe_load(Path("../config/params.yaml").read_text())
tickers = params["universe"]["tickers"]
period = params["data"]["period"]
interval = params["data"]["interval"]
field = params["data"]["field"]
tz = params["data"]["tz"]

path_close = params["paths"]["processed_close"]
path_returns = params["paths"]["processed_returns"]
path_corr = params["paths"]["corr"]

In [11]:
raw = yf.download(tickers, period=period, interval=interval, auto_adjust=False)
closes = extract_field_yf(raw, field=field)
prices = prepare_prices(closes, tz=tz)
prices.head()

[*********************100%***********************]  9 of 9 completed


Ticker,BAC,BK,C,GS,JPM,MS,PNC,USB,WFC
Datetime,,,,,,,,,
2023-10-18 15:30:00+02:00,27.920000,43.369999,40.639999,305.484985,146.850006,74.910004,117.120003,33.884998,41.669998
2023-10-18 16:30:00+02:00,27.546000,43.435001,40.509899,303.239990,146.389999,74.190002,116.500000,33.733002,41.665001
2023-10-18 17:30:00+02:00,27.520000,43.259998,40.369999,302.864990,146.199997,74.084999,116.356400,33.744999,41.549999
2023-10-18 18:30:00+02:00,27.565001,43.430000,40.445000,303.122314,146.690002,74.570099,116.980003,33.759998,41.555000
2023-10-18 19:30:00+02:00,27.480000,43.389999,40.419998,303.279999,146.399994,74.650002,116.754997,33.717999,41.509998


In [12]:
returns = compute_returns(prices, log=True)
corr = returns.corr()

save_parquet(prices, f"../{path_close}")
save_parquet(returns, f"../{path_returns}")
save_parquet(corr, f"../{path_corr}")

print("✅ Saved:")
print(f"- close -> {path_close}")
print(f"- returns -> {path_returns}")
print(f"- corr -> {path_corr}")

✅ Saved:
- close -> data/processed/close_1h.parquet
- returns -> data/processed/returns_1h.parquet
- corr -> data/processed/corr_1h.parquet
